# Sentiment Analysis with Deep Learning using BERT

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Project Outline

**Task 1**: Introduction (this section)

**Task 2**: Exploratory Data Analysis and Preprocessing

**Task 3**: Training/Validation Split

**Task 4**: Loading Tokenizer and Encoding our Data

**Task 5**: Setting up BERT Pretrained Model

**Task 6**: Creating Data Loaders

**Task 7**: Setting Up Optimizer and Scheduler

**Task 8**: Defining our Performance Metrics

**Task 9**: Creating our Training Loop

**Task 10**: Loading and Evaluating our Model

## Task 1: Introduction

<img src="Images/BERT_diagrams.pdf" width="1000">

## Task 2: Exploratory Data Analysis and Preprocessing

We will use the SMILE Twitter dataset.

_Wang, Bo; Tsakalidis, Adam; Liakata, Maria; Zubiaga, Arkaitz; Procter, Rob; Jensen, Eric (2016): SMILE Twitter Emotion dataset. figshare. Dataset. https://doi.org/10.6084/m9.figshare.3187909.v2_

In [3]:
import torch
import pandas as pd
from tqdm.notebook import tqdm

In [4]:
df = pd.read_csv('drive/MyDrive/bert_nlp/smile-annotations-final.csv',
                 names = ['id','text','category'])
df.set_index('id',inplace=True)

In [5]:
df.head()

,text,category
id,,
6.121680e+17,@stiveshouse @MillenniumArt @Tate_StIves @Trem...,happy
6.116340e+17,I miss Cornwall so much. #ff @FernPitCafe @Pen...,not-relevant
6.107630e+17,£10 off @accessart http://t.co/0biFQaXTQg memb...,not-relevant
6.115290e+17,@britishmuseum @Ophiolatrist Spain? Egypt?,nocode
6.151070e+17,Enjoying the amazing collections @britishmuseu...,happy


In [6]:
df.text.iloc[1]

'I miss Cornwall so much. #ff @FernPitCafe @PentireNewquay @FalconInnMawgan @trebahgarden @Tate_StIves @NewquayRNLI @HeliganGardens'

In [7]:
df.category.value_counts()

nocode               1554
happy                1123
not-relevant          211
angry                  56
surprise               35
sad                    32
happy|surprise         11
happy|sad               9
disgust|angry           7
disgust                 6
sad|disgust             2
sad|angry               2
sad|disgust|angry       1
Name: category, dtype: int64

In [8]:
df = df[~df.category.str.contains('\|')]
df = df[df.category != 'nocode']
df.category.value_counts()

happy           1123
not-relevant     211
angry             56
surprise          35
sad               32
disgust            6
Name: category, dtype: int64

In [9]:
labels = df.category.unique()
labels

array(['happy', 'not-relevant', 'angry', 'disgust', 'sad', 'surprise'],
      dtype=object)

In [10]:
label_dict = {}
for index, label in enumerate(labels):
    label_dict[label] = index
label_dict

{'angry': 2,
 'disgust': 3,
 'happy': 0,
 'not-relevant': 1,
 'sad': 4,
 'surprise': 5}

In [11]:
df['label'] = df.category.replace(label_dict)
df.head()

,text,category,label
id,,,
6.121680e+17,@stiveshouse @MillenniumArt @Tate_StIves @Trem...,happy,0
6.116340e+17,I miss Cornwall so much. #ff @FernPitCafe @Pen...,not-relevant,1
6.107630e+17,£10 off @accessart http://t.co/0biFQaXTQg memb...,not-relevant,1
6.151070e+17,Enjoying the amazing collections @britishmuseu...,happy,0
6.112390e+17,@NationalGallery Beautiful painting. Thank you.,happy,0


## Task 3: Training/Validation Split

In [12]:
from sklearn.model_selection import train_test_split

In [13]:
X_train, X_val, y_train, y_val = train_test_split(
    df.index.values,
    df.label.values,
    test_size = 0.15,
    random_state = 17,
    stratify = df.label.values
)

In [14]:
df['data_type'] = ['not_set']*df.shape[0]
df.head()

,text,category,label,data_type
id,,,,
6.121680e+17,@stiveshouse @MillenniumArt @Tate_StIves @Trem...,happy,0,not_set
6.116340e+17,I miss Cornwall so much. #ff @FernPitCafe @Pen...,not-relevant,1,not_set
6.107630e+17,£10 off @accessart http://t.co/0biFQaXTQg memb...,not-relevant,1,not_set
6.151070e+17,Enjoying the amazing collections @britishmuseu...,happy,0,not_set
6.112390e+17,@NationalGallery Beautiful painting. Thank you.,happy,0,not_set


In [15]:
df.loc[X_train,'data_type'] = 'train'
df.loc[X_val,'data_type'] = 'val'
df.head()

,text,category,label,data_type
id,,,,
6.121680e+17,@stiveshouse @MillenniumArt @Tate_StIves @Trem...,happy,0,train
6.116340e+17,I miss Cornwall so much. #ff @FernPitCafe @Pen...,not-relevant,1,train
6.107630e+17,£10 off @accessart http://t.co/0biFQaXTQg memb...,not-relevant,1,train
6.151070e+17,Enjoying the amazing collections @britishmuseu...,happy,0,val
6.112390e+17,@NationalGallery Beautiful painting. Thank you.,happy,0,train


In [16]:
df.groupby(['category','label','data_type']).count()

text
category     label data_type      
angry        2     train        41
                   val          15
disgust      3     train         4
                   val           2
happy        0     train       872
                   val         251
not-relevant 1     train       160
                   val          51
sad          4     train        24
                   val           8
surprise     5     train        24
                   val          11

## Task 4: Loading Tokenizer and Encoding our Data

In [17]:
!pip install transformers
from transformers import BertTokenizer
from torch.utils.data import TensorDataset

     |████████████████████████████████| 2.9 MB 7.2 MB/s 
     |████████████████████████████████| 895 kB 56.2 MB/s 
     |████████████████████████████████| 636 kB 63.8 MB/s 
     |████████████████████████████████| 3.3 MB 53.2 MB/s 
     |████████████████████████████████| 52 kB 1.9 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [18]:
tokenizer = BertTokenizer.from_pretrained(
    'bert-base-uncased',
    do_lower_case = True,
)

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [19]:
encoded_data_train = tokenizer.batch_encode_plus(
    df[df.data_type=='train'].text.values,
    add_special_tokens=True,
    return_attention_mask = True,
    pad_to_max_length = True,
    max_length = 256,
    return_tensors='pt'
)

encoded_data_val = tokenizer.batch_encode_plus(
    df[df.data_type=='val'].text.values,
    add_special_tokens=True,
    return_attention_mask = True,
    pad_to_max_length = True,
    max_length = 256,
    return_tensors='pt'
)

input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(df[df.data_type=='train'].label.values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(df[df.data_type=='val'].label.values)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2217: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [20]:
dataset_train = TensorDataset(
    input_ids_train,
    attention_masks_train,
    labels_train
)
dataset_val = TensorDataset(
    input_ids_val,
    attention_masks_val,
    labels_val
)

In [21]:
len(dataset_train)

1125

In [22]:
len(dataset_val)

338

## Task 5: Setting up BERT Pretrained Model

In [23]:
from transformers import BertForSequenceClassification

In [24]:
model = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased',
    num_labels = len(label_dict),
    output_attentions = False,
    output_hidden_states = False
)

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

## Task 6: Creating Data Loaders

In [25]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

In [26]:
batch_size = 32

dataloader_train = DataLoader(
    dataset_train,
    sampler = RandomSampler(dataset_train),
    batch_size=batch_size
)

dataloader_val = DataLoader(
    dataset_val,
    sampler = RandomSampler(dataset_val),
    batch_size=batch_size
)

## Task 7: Setting Up Optimizer and Scheduler

In [27]:
from transformers import AdamW, get_linear_schedule_with_warmup

In [28]:
optimizer = AdamW(
    model.parameters(),
    lr = 1e-5,
    eps = 1e-8
)

In [29]:
epochs = 10
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps = 0,
    num_training_steps = len(dataloader_train)*epochs
)

## Task 8: Defining our Performance Metrics

Accuracy metric approach originally used in accuracy function in [this tutorial](https://mccormickml.com/2019/07/22/BERT-fine-tuning/#41-bertforsequenceclassification).

In [30]:
import numpy as np
from sklearn.metrics import f1_score

In [31]:
def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds,axis = 1).flatten()
    labels_flat = labels.flatten()
    return f1_score(
        labels_flat,
        preds_flat,
        average = 'weighted'
    )

In [32]:
def accuracy_per_class(preds, labels):
    label_dict_inverse = {v:k for k, v in label_dict.items()}
    
    preds_flat = np.argmax(preds,axis = 1).flatten()
    labels_flat = labels.flatten()
    
    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')

## Task 9: Creating our Training Loop

Approach adapted from an older version of HuggingFace's `run_glue.py` script. Accessible [here](https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128).

In [33]:
import random

seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [34]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

print(device)

cuda


In [35]:
def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in tqdm(dataloader_val):
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals

In [36]:
import torch.nn.utils
for epoch in tqdm(range(1, epochs+1)):
    
    model.train()
    
    loss_train_total = 0
    
    progress_bar = tqdm(
        dataloader_train,
        desc ='Epoch {:1d}'.format(epoch),
        leave = False,
        disable = False
    )
    
    for batch in progress_bar:
        model.zero_grad()
        batch = tuple(b.to(device) for b in batch)
        inputs = {
            'input_ids'     :batch[0],
            'attention_mask':batch[1],
            'labels'        :batch[2]
        }
        outputs = model(**inputs)
        
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(),1.0)
        
        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss':'{:.3f}'.format(loss.item()/len(batch))})
    
    torch.save(model.state_dict(),f'drive/MyDrive/bert_nlp/Models/BERT_ft_epoch{epoch}.model')
    tqdm.write(f'\nEpoch {epoch}')
    loss_train_avg = loss_train_total/len(dataloader_train)
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(dataloader_val)
    val_f1 = f1_score_func(predictions,true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (weighted): {val_f1}')

  0%|          | 0/10 [00:00<?, ?it/s]

Epoch 1:   0%|          | 0/36 [00:00<?, ?it/s]


Epoch 1
Training loss: 1.3098017788595624


  0%|          | 0/11 [00:00<?, ?it/s]

Validation loss: 0.9169477982954546
F1 Score (weighted): 0.6329150802182014


...

Validation loss: 0.5876803858713671
F1 Score (weighted): 0.7929692851122893


Epoch 10:   0%|          | 0/36 [00:00<?, ?it/s]


Epoch 10
Training loss: 0.3200438531736533


  0%|          | 0/11 [00:00<?, ?it/s]

Validation loss: 0.5927881273356351
F1 Score (weighted): 0.7977735047195438


## Task 10: Loading and Evaluating our Model

In [37]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [38]:
model.to(device)
pass

In [39]:
model.load_state_dict(
    torch.load('drive/MyDrive/bert_nlp/Models/BERT_ft_epoch10.model')
)

<All keys matched successfully>

In [40]:
_, predictions,true_vals = evaluate(dataloader_val)

  0%|          | 0/11 [00:00<?, ?it/s]

In [41]:
accuracy_per_class(predictions, true_vals)

Class: happy
Accuracy: 248/251

Class: not-relevant
Accuracy: 29/51

Class: angry
Accuracy: 5/15

Class: disgust
Accuracy: 0/2

Class: sad
Accuracy: 0/8

Class: surprise
Accuracy: 0/11

